<a href="https://colab.research.google.com/github/heysayanallgood/GDSCPython/blob/Bot/DiscordBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U discord.py[voice] pynacl nest_asyncio youtube_dl transformers google-generativeai apscheduler yt-dlp
!apt install ffmpeg -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installe

In [ ]:
import nest_asyncio
nest_asyncio.apply()

import discord
from discord.ext import commands
from datetime import datetime, timedelta
from apscheduler.schedulers.asyncio import AsyncIOScheduler
import yt_dlp as youtube_dl
from collections import deque
from transformers import pipeline
import google.generativeai as genai
import asyncio

# Initialize bot
intents = discord.Intents.default()
intents.message_content = True
intents.members = True
intents.voice_states = True

bot = commands.Bot(command_prefix="!", intents=intents)

# Gemini API setup
# IMPORTANT: Replace with your actual API key
GEMINI_API_KEY = 'YOUR_GEMINI_KEY'
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

# Scheduler for reminders
scheduler = AsyncIOScheduler()
reminders = {}

# Music queue
music_queue = deque()

# Summarization pipeline
try:
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
except Exception as e:
    print(f"Summarization model load error: {e}")
    summarizer = None

# YouTube DL configuration
ytdl_format_options = {
    'format': 'bestaudio/best',
    'noplaylist': True,
    'quiet': True,
    'no_warnings': True,
    'default_search': 'auto',
    'source_address': '0.0.0.0'
}

ffmpeg_options = {
    'options': '-vn',
    'before_options': '-reconnect 1 -reconnect_streamed 1 -reconnect_delay_max 5'
}

ytdl = youtube_dl.YoutubeDL(ytdl_format_options)

class YTDLSource(discord.PCMVolumeTransformer):
    def __init__(self, source, *, data, volume=0.5):
        super().__init__(source, volume)
        self.data = data
        self.title = data.get('title')
        self.url = data.get('url')

    @classmethod
    async def from_url(cls, url, *, loop=None, stream=True):
        loop = loop or asyncio.get_event_loop()
        data = await loop.run_in_executor(None, lambda: ytdl.extract_info(url, download=not stream))

        if 'entries' in data:
            data = data['entries'][0]

        filename = data['url'] if stream else ytdl.prepare_filename(data)
        return cls(discord.FFmpegPCMAudio(filename, **ffmpeg_options), data=data)

# Bot Events
@bot.event
async def on_ready():
    print(f'Logged in as {bot.user}')
    print(f'Bot ID: {bot.user.id}')
    scheduler.start()

@bot.event
async def on_member_join(member):
    channel = member.guild.system_channel or discord.utils.get(member.guild.text_channels, name="general")
    if channel:
        await channel.send(f"Welcome {member.mention} to the server! 🎉")

# Commands
@bot.command(name='chat')
async def chat(ctx, *, message: str):
    try:
        # Show typing indicator
        async with ctx.typing():
            # Use async-friendly generation method
            response = await asyncio.to_thread(
                model.generate_content,
                message
            )

            # Check if response is valid
            if not response or not response.text:
                await ctx.send("❌ No response generated.")
                return

            # Send response, truncating to Discord's message limit
            await ctx.send(response.text[:2000])
    except Exception as e:
        await ctx.send(f"❌ Chat error: {str(e)}")

@bot.command(name='remind')
async def remind(ctx, time: str, date: str, *, message: str):
    try:
        reminder_time = datetime.strptime(f"{date} {time}", '%Y-%m-%d %H:%M')
        if reminder_time < datetime.now():
            await ctx.send("⏰ Please provide a future time!")
            return

        job = scheduler.add_job(
            send_reminder,
            'date',
            run_date=reminder_time,
            args=[ctx.author.id, message]
        )
        reminders[job.id] = message
        await ctx.send(f"✅ Reminder set for {reminder_time.strftime('%Y-%m-%d %H:%M')}")
    except Exception as e:
        await ctx.send(f"❌ Invalid format. Use: !remind HH:MM YYYY-MM-DD <message>")

async def send_reminder(user_id, message):
    try:
        user = await bot.fetch_user(user_id)
        await user.send(f"⏰ Reminder: {message}")
    except:
        pass

@bot.command(name='poll')
async def poll(ctx, question: str, *options: str):
    if len(options) < 2 or len(options) > 10:
        await ctx.send("❌ Please provide 2-10 options")
        return

    emojis = ['1️⃣', '2️⃣', '3️⃣', '4️⃣', '5️⃣', '6️⃣', '7️⃣', '8️⃣', '9️⃣', '🔟']
    description = "\n".join([f"{emojis[idx]} {option}" for idx, option in enumerate(options)])
    embed = discord.Embed(title=question, description=description)
    message = await ctx.send(embed=embed)
    for emoji in emojis[:len(options)]:
        await message.add_reaction(emoji)

@bot.command(name='join')
async def join(ctx):
    try:
        if not ctx.author.voice:
            await ctx.send("❌ You're not in a voice channel!")
            return

        channel = ctx.author.voice.channel

        if ctx.voice_client:
            await ctx.voice_client.move_to(channel)
        else:
            await channel.connect()

        await ctx.send(f"✅ Joined {channel.name}")
    except Exception as e:
        await ctx.send(f"❌ Error: {str(e)}")

@bot.command(name='play')
async def play(ctx, *, url: str):
    try:
        # Ensure bot is in voice channel
        if not ctx.voice_client:
            await ctx.invoke(join)

        async with ctx.typing():
            # Add more robust error handling
            try:
                player = await YTDLSource.from_url(url, loop=bot.loop, stream=True)
            except Exception as ydl_error:
                await ctx.send(f"❌ YouTube download error: {str(ydl_error)}")
                return

            # Improved queue and playback management
            def after_playing(error):
                if error:
                    print(f'Player error: {error}')
                bot.loop.create_task(play_next(ctx))

            if ctx.voice_client.is_playing():
                music_queue.append(player)
                await ctx.send(f"🎵 Added to queue: {player.title}")
            else:
                ctx.voice_client.play(player, after=after_playing)
                await ctx.send(f"🎶 Now playing: {player.title}")
    except Exception as e:
        await ctx.send(f"❌ Playback error: {str(e)}")

async def play_next(ctx):
    if music_queue and ctx.voice_client:
        if not ctx.voice_client.is_playing():
            player = music_queue.popleft()
            ctx.voice_client.play(player, after=lambda e: bot.loop.create_task(play_next(ctx)))
            await ctx.send(f"🎶 Now playing: {player.title}")

@bot.command(name='stop')
async def stop(ctx):
    if ctx.voice_client:
        await ctx.voice_client.disconnect()
        music_queue.clear()
        await ctx.send("🛑 Stopped playing and cleared queue")

@bot.command(name='summarize')
async def summarize(ctx, *, text: str):
    try:
        # Check if summarizer is loaded
        if not summarizer:
            await ctx.send("❌ Summarization model not available")
            return

        # More robust text length checking
        if len(text.split()) < 50:
            await ctx.send("❌ Text must be at least 50 words long to summarize")
            return

        # Async summarization
        summary = await asyncio.to_thread(
            summarizer,
            text,
            max_length=150,
            min_length=30,
            do_sample=False
        )

        await ctx.send(f"📝 Summary: {summary[0]['summary_text']}")
    except Exception as e:
        await ctx.send(f"❌ Summarization error: {str(e)}")

# Bot Token
# IMPORTANT: Replace with your actual Discord bot token
BOT_TOKEN = 'YOUR_DISCORD_TOKEN'

# Main bot runner
async def main():
    async with bot:
        await bot.start(BOT_TOKEN)

# Run the bot
if __name__ == '__main__':
    asyncio.run(main())

Device set to use cpu


Logged in as projectbot#1508
Bot ID: 1346154080094916629


KeyboardInterrupt: 